# =============================================================
# MILESTONE 3: Machine Learning Model Development and Optimization
# =============================================================

In [4]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

In [5]:
path=r"C:\Users\starr\customer-churn-prediction-analysis\data\processed\cleaned_data.csv"
df = pd.read_csv(path) 

# Features (X) and Target (y)

In [6]:
X = df.drop("Churn", axis=1)
y = df["Churn"]

# Train/test split (80/20) with stratification

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Before SMOTE:", np.bincount(y_train))

Before SMOTE: [1822  310]



# Apply SMOTE only to training set

In [8]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("After SMOTE:", np.bincount(y_train_resampled))

After SMOTE: [1822 1822]
